In [1]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_KEY")
deployment = os.environ.get("AZURE_OPENAI_MODEL")

In [21]:
client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01"
)

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "hur anmäler jag ett brott??",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.environ["AZURE_SEARCH_ENDPOINT"],
                    "index_name": "test1-new",
                    "authentication": {
                        "type": "api_key",
                        "key": os.environ["AZURE_SEARCH_KEY"],
                    },
                    "fields_mapping": {
                            "content_fields": ["content", "testField"],
                            "title_field": "title",
                            "url_field": "url",
                            "filepath_field": "filepath",
                            "vector_fields": ["contentVector"],
                            "content_fields_separator": "asdqwe123"
                    }
                }
            }
        ],
    }
)

print(completion.model_dump_json(indent=2))

{
  "id": "449818c9-bf59-48fb-8458-05a256c84e44",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The retrieved document provides information on how to report a crime to the police. It is intended for all activities and managers within the Västerbotten Region when reporting a crime to the police. The document contains an external link to the police for making a report of a crime, theft, etc. [doc1].",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "end_turn": true,
        "context": {
          "citations": [
            {
              "content": "Dokumentet gäller inom Plats: Alla Org: Region Västerbotten\nUtgåva nr 2 Giltigt fro.m: 2019-10-30 - Tills vidare Uppföljning : 2022-01-04\nVÄSTERBOTTENS LÄNS LANDSTING\nStyrande dokument Externlänk\n<h2>Anmälan om brott till polisen</h2>\nOmfattning\nDet här dokumentet riktar sig till alla verksamheter och c

In [23]:
completion.choices[0].message.context['citations'][0]['content'].split("asdqwe123")

['Dokumentet gäller inom Plats: Alla Org: Region Västerbotten\nUtgåva nr 2 Giltigt fro.m: 2019-10-30 - Tills vidare Uppföljning : 2022-01-04\nVÄSTERBOTTENS LÄNS LANDSTING\nStyrande dokument Externlänk\n<h2>Anmälan om brott till polisen</h2>\nOmfattning\nDet här dokumentet riktar sig till alla verksamheter och chefer inom Region Västerbotten när verksamheten ska göra en polisanmälan.\nBeskrivning\nDetta dokument är en extern länk till polisen när verksamheten ska göra en anmälan om brott, stöld mm till polisen.\nhttps://anmalan.polisen.se/imse/foremal',
 'this is a test field']